In [4]:
from fastdownload import download_url
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.all import *
from duckduckgo_search import ddg_images

In [5]:
def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')
searches = 'dragonfly','spider'


In [6]:
path = Path('dragonfly_or_not')

In [7]:
for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photos', 100))
    resize_images(path/o, max_size=400, dest=path/o)

Searching for 'dragonfly photos'
Searching for 'spider photos'


In [8]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

2

In [9]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

In [10]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)


/home/rati/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rati/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


epoch,train_loss,valid_loss,error_rate,time
0,1.032743,0.199212,0.081081,00:03


epoch,train_loss,valid_loss,error_rate,time
0,0.351028,0.153600,0.054054,00:03
1,0.207677,0.137419,0.054054,00:03
2,0.170126,0.179732,0.054054,00:03
3,0.137768,0.217194,0.054054,00:03


In [12]:
is_dragonfly, _, probs = learn.predict(PILImage.create('spider.jpg'))
print(f"This is a: {is_dragonfly}.")
print(f"Probability it's a dragonfly: {probs[0]:.4f}")

This is a: spider.
Probability it's a dragonfly: 0.0001
